# Implementierung von Präfix-Baum und Radix-Tree (PATRICIA-Trie)
Basierend auf http://christianherta.de/lehre/informationRetrieval/trie.php

## Präfix-Baum Interface
Die *delete-Methode* ist als Übung zu implementieren.

In [11]:
import abc

class TrieI(object):
    
    __metaclass__ = abc.ABCMeta
    
    @abc.abstractmethod
    def insert(self, k, v):
        """
            Insert a key and a value into the Trie.           
        """
        return
    
    @abc.abstractmethod
    def has_key(self, k):
        """
            Check if the key was already inserted into the trie. Should return a boolean value.
        """
        return
    
    @abc.abstractmethod
    def get_value(self, k):
        """
            Get the value for the key
            If the key is not present, return None.
        """
        return
    
    @abc.abstractmethod
    def start_with_prefix(self, prefix):
        """
            Give all keys (as list) with the given prefix.
            If there is no such a prefix in the trie, return empty list.
        """
        return
    
    # exercise:
    #@abc.abstractmethod
    #def delete(self, key):
        #"""
        #    Delete the Key-Value Pair from the Trie
        #"""
        #return
    

## Implementierung des Präfix-Baum
Default-mäßig werden als Schlüssel des Präfix-Baum Zeichenketten verwendet. Durch eine entsprechende Definition der *key_concatenation_func* können auch andere sequentielle Datenstrukturen (wie z.B. Listen) als Schlüssel verwendet werden.

Die *delete-methode* ist als Übung selbst zu implementieren.

In [12]:
class TrieNode(TrieI):
    
    def __init__(self, k="", key_concatenation_func=None):
        """
            :type k: has to be the type of the key 
            :param k: empty string or list
            :type key_concatenation_func: function
            :param key_concatenation_func: operation for concatenation of the key-elements to reconstruct
                                              the key from it's elementes
        """
        self._children = {}
        self._value = None 
        self.k_init = k
        if key_concatenation_func is None:
            # standard keys are strings
            self.key_concatenation_func = self._strjoin
        else:
            self.key_concatenation_func = key_concatenation_func
    
    def _strjoin(self,a, b):
        return "".join([a,b])
    
    # standard behaviour for the insertion of a value:
    def _update_value(self, v):
        self._value = v
        
    def insert(self, k, v):     
        if len(k) == 0: 
            self._update_value(v)
            return
        if k[0] not in self._children: 
           # type(self) get's the current type for calling the appropriate constructor in subclasses
           self._children[k[0]] = type(self)(self.k_init , key_concatenation_func=self.key_concatenation_func)
        self._children[k[0]].insert(k[1:], v)
            
    def has_key(self, k):
        if len(k) == 0:
            if self._value is not None:
                return True
            else:
                return False
        if k[0] not in self._children:
            return False
        return self._children[k[0]].has_key(k[1:])
    
    def get_value(self, k):
        if len(k) == 0 :
            return self._value
        if k[0] not in self._children:
            return None
        return self._children[k[0]].get_value(k[1:])
    
    def _subtree_keys(self, result, key):
        if self._value is not None:
            result.append(key)
        for k, v in self._children.items():
            result = v._subtree_keys(result, self.key_concatenation_func(key, k))
        return result
    
    def start_with_prefix(self, prefix):
        return self._start_with_prefix(prefix, prefix)
    
    def _start_with_prefix(self, prefix, key=None):
        # collect all subtree nodes
        if len(prefix) == 0: 
            return self._subtree_keys([], key)
        if prefix[0] not in self._children:
            return []
        # decend down tree until prefix is exhausted
        return self._children[prefix[0]]._start_with_prefix(prefix[1:], key)

In [13]:
trie = TrieNode()
trie.insert('karl', 10)
trie.insert('karlchen', 13)
trie.insert('karlson', 3)
trie.insert('karlsons', 7)
trie.insert('something', 1)

assert trie.has_key('karlchen') is True
assert trie.get_value('karlchen') == 13
assert trie.has_key('nothing') is False
assert trie.get_value('nothing') == None

print(trie.start_with_prefix('kar'))
#print(trie.start_with_prefix('karls'))
#print(trie.start_with_prefix(''))
#print(trie.start_with_prefix('nothing'))


['karl', 'karlson', 'karlsons', 'karlchen']


In [14]:
trie._subtree_keys([], '')

['something', 'karl', 'karlson', 'karlsons', 'karlchen']

## Implementierung des Radix-Tree
Default-mäßig werden als Schlüssel des Radix-Tree Zeichenketten verwendet. 

Um andere sequentielle Datenstrukturen (wie z.B. Listen) als Schlüssel zu verwenden sind sowohl die *key_concatenation_func* zu definieren, als auch alle Vorkommen von *k.startswith(self._prefix)* entsprechend anzupassen.

Die *delete-methode* ist als Übung selbst zu implementieren.

In [15]:
class RadixTreeNode(TrieNode):
    
    nodes = 0
    
    def __init__(self, k="", key_concatenation_func=None):
        """
        Implements a Radix Tree on the basis of TrieNode
            :type k: has to be the type of the key 
            :param k: empty string or list
            :type key_concatenation_func: function
            :param key_concatenation_func: operation for concatenation of the key-elements to reconstruct
                                              the key from it's elementes
        Instead of maintaining a single dictionary _children for the next element, its _prefix variable
        contains the prefix common to all children and _children just a map to the next differing elements.
        Thus, _prefix can contain longer sequences of elements which are common to all of its children. In the
        worts case (all children differ), it is equivalent to a TrieNode.
        """
        
        self._children = {}
        self._value = None
        self._prefix = None
        self.k_init = k
        if key_concatenation_func is None:
            # standard keys are strings
            self.key_concatenation_func = self._strjoin
        else:
            self.key_concatenation_func = key_concatenation_func
            
    def _strjoin(self,a, b):
        return "".join([a,b])
    
    # greatest common prefix
    def _greatest_common_prefix(self, k):
        # return gcs and index of remaining substring
        i = 0
        while i < min(len(k),len(self._prefix)) and k[i] == self._prefix[i]:
            i += 1
        return (self._prefix[:i],i)
           
    def insert(self, k, v):     
        if len(k) == 0: 
            self._update_value(v)
            return
        if self._prefix == None or self._prefix == k:
            self._prefix = k
            self._value = v
            return
        (gcs,i) = self._greatest_common_prefix(k)
        if gcs == self._prefix: 
            if k[i] not in self._children: 
                # type(self) get's the current type for calling the appropriate constructor in subclasses
                self._children[k[i]] = type(self)(self.k_init , key_concatenation_func=self.key_concatenation_func)
            self._children[k[i]].insert(k[i:], v)
        else:
            # gcs is smaller than _prefix
            # create new child
            newNode = type(self)(self.k_init , key_concatenation_func=self.key_concatenation_func)
            # push down self's _children to the new node
            newNode._children = self._children
            # push down value to new node
            newNode._value = self._value
            # init new nodes prefix with rest of own prefix
            newNode._prefix = self._prefix[i:]
            # new node becomes child of self at first diverging element
            self._children = dict({self._prefix[i]: newNode})
            # update _prefix with new gcd
            self._prefix = gcs
            # if the gcs equals the key, we need to set the value of the node, since the string is exhaused, 
            # otherwise the value needs to be resetto None, since partial string was never seen before
            self._value = v if gcs == k else None
            if i < len(k):
                # recurse down and set branching char
                if k[i] not in self._children: 
                    # type(self) get's the current type for calling the appropriate constructor in subclasses
                    self._children[k[i]] = type(self)(self.k_init , key_concatenation_func=self.key_concatenation_func)
                self._children[k[i]].insert(k[i:], v)
                
    def has_key(self, k):
        if self._prefix == None:
            return False
        if len(k) == 0 or k == self._prefix:
            if self._value is not None:
                return True
            else:
                return False
        elif k.startswith(self._prefix):
            i = len(self._prefix)
            if k[i] not in self._children:
                return False
            return self._children[k[i]].has_key(k[i:])
        elif not k.find(self._prefix):
            if k[0] not in self._children:
                return False
            return self._children[k[0]].has_key(k)
        else:
            return False
            
    def get_value(self, k):
        if len(k) == 0 or k == self._prefix:
            return self._value
        if k.startswith(self._prefix):
            i = len(self._prefix)
            if k[i] not in self._children:
                return None
            return self._children[k[i]].get_value(k[i:])
        elif not k.find(self._prefix):
            if k[0] not in self._children:
                return None
            return self._children[k[0]].get_value(k)
        else:
            return None   
 
    def _subtree_keys(self, result, key):
        if self._value and not key == '':
            result.append(key)
        for k, v in self._children.items():
            result = v._subtree_keys(result, self.key_concatenation_func(key, v._prefix))
        return result
    
    def start_with_prefix(self, prefix):
        # this returns only proper keys, which are included in the trie
        return self._start_with_prefix(prefix, prefix)

    def _start_with_prefix(self, k, key):
        # collect all subtree nodes
        if len(k) == 0 or k == self._prefix:
            return self._subtree_keys([], key)
        if k.startswith(self._prefix):
            i = len(self._prefix)
            if k[i] not in self._children:
                return []
            return self._children[k[i]]._start_with_prefix(k[i:], key)
        elif self._prefix.startswith(k):
            return self._subtree_keys([], self.key_concatenation_func(key[:-len(k)], self._prefix))
        elif not k.find(self._prefix):
            if k[0] not in self._children:
                return None
            # return self._children[k[0]]._start_with_prefix(k,key)
        else:
            return [] 

In [16]:
trie = RadixTreeNode()
trie.insert('karl', 10)
trie.insert('karlchen', 13)
trie.insert('karlson', 3)
trie.insert('karlsons', 7)
trie.insert('something', 1)

assert trie.has_key('karlchen') is True
assert trie.get_value('karlchen') == 13
assert trie.has_key('nothing') is False
assert trie.get_value('nothing') == None

print(trie.start_with_prefix('karl'))
print(trie.start_with_prefix('karlc'))
print(trie.start_with_prefix('karlson'))
print(trie.start_with_prefix(''))
print(trie.start_with_prefix('nothing'))
print(trie.start_with_prefix('s'))


['karl', 'karlson', 'karlsons', 'karlchen']
['karlchen']
['karlson', 'karlsons']
['something', 'karl', 'karlson', 'karlsons', 'karlchen']
[]
['something']


In [17]:
trie.get_value('karl')

10

Die Implementierung kann nicht nur mit String-Keys umgehen, sondern auch mir anderen Listen als Keys. Dem Konstruktor muss hierfür eine leeres Listen-Object und eine entsprechende key_concatenation_func übergeben werden. 

In [18]:
def append_to_list(a, b):
    x = list(a)
    x.append(b)
    return x

ngram_trie = RadixTreeNode(k=[], key_concatenation_func=append_to_list)

ngram_trie.insert(["ein","tag", "im"], 4) 
ngram_trie.insert(["ein","tag", "im", "mai"], 1) 

assert ngram_trie.get_value(["ein","tag", "im", "mai"]) == 1 
assert ngram_trie.has_key(["ein","tag", "im"]) is True
assert ngram_trie.has_key(["tag", "im", "mai"]) is False

print(ngram_trie.start_with_prefix(['ein','tag']))

AttributeError: 'list' object has no attribute 'startswith'

## Beispiel: Anwendung als Termlexikon

Um die Trie-Implementierung als Term-Lexikon zu verwenden, kann die _update_value-Methode überschrieben werden. In diesem Beispiel soll die Korpusfrequenz (cf) als Value gespeichert werden.

In [19]:
class TrieLexicon(TrieNode):
    
    #def __init__(self, k="", key_concatenation_func=None):
    #    super(TrieLexicon, self).__init__(k, key_concatenation_func)
    
    def _update_value(self, v):
        """
         overwrite default behaviour
        """
        if self._value == None:
            self._value = v
        else: 
            self._value += v 

In [20]:
from nltk.corpus import brown
brown_news_words = brown.words(categories='news')

lexicon = TrieLexicon()

for w in brown_news_words[0:3000] :
    # here we can do some normalization
    lexicon.insert(w, 1)
    
for k in lexicon.start_with_prefix("ad"):
    print(k, ": ", lexicon.get_value(k)) 

adamant :  1
advocacy :  1
administration :  2
administrators :  1
adjournment :  2
adjustments :  1
additional :  2
added :  3
